To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


# Introduction

<center><img src="https://cdn.mos.cms.futurecdn.net/RNrVwVfRiyoKkrr8djHvf9-650-80.jpg.webp"></center>
<center><font color="BBBBBB" size=2>(Image credit: Adobe Firefly - AI generated for Future, from Tom's guide)</font></center>

## Objective

Use Llama3 Langchain and ChromaDB to create a Retrieval Augmented Generation (RAG) system. This will allow us to ask questions about our documents (that were not included in the training data), without fine-tunning the Large Language Model (LLM).
When using RAG, if you are given a question, you first do a retrieval step to fetch any relevant documents from a special database, a vector database where these documents were indexed.
The data that we will use is the text of EU AI Act, approved on March 13, 2024.

## Definitions

* LLM - Large Language Model  
* Llama3- LLM from Meta
* Langchain - a framework designed to simplify the creation of applications using LLMs
* Vector database - a database that organizes data through high-dimmensional vectors  
* ChromaDB - vector database  
* RAG - Retrieval Augmented Generation (see below more details about RAGs)

## Model details

* **Model**: Llama 3  
* **Variation**: 8b-chat-hf  (8b: 8B dimm.; hf: HuggingFace)
* **Version**: V1  
* **Framework**: Transformers  

Llama3 model is pretrained and fine-tuned with 15T+ (more than 15 Trillion) tokens and 8 to 70 Billion parameters which makes it one of the powerful open source models. It is a highly improvement over Llama2 model.


## What is a Retrieval Augmented Generation (RAG) system?

Large Language Models (LLMs) has proven their ability to understand context and provide accurate answers to various NLP tasks, including summarization, Q&A, when prompted. While being able to provide very good answers to questions about information that they were trained with, they tend to hallucinate when the topic is about information that they do "not know", i.e. was not included in their training data. Retrieval Augmented Generation combines external resources with LLMs. The main two components of a RAG are therefore a retriever and a generator.  

The retriever part can be described as a system that is able to encode our data so that can be easily retrieved the relevant parts of it upon queriying it. The encoding is done using text embeddings, i.e. a model trained to create a vector representation of the information. The best option for implementing a retriever is a vector database. As vector database, there are multiple options, both open source or commercial products. Few examples are ChromaDB, Mevius, FAISS, Pinecone, Weaviate. Our option in this Notebook will be a local instance of ChromaDB (persistent).

For the generator part, the obvious option is a LLM. In this Notebook we will use a quantized Llama3 model, from the Kaggle Models collection.  

The orchestration of the retriever and generator will be done using Langchain. A specialized function from Langchain allows us to create the receiver-generator in one line of code.

## The data

The data that will be indexed in the vector database to make it searchable by the RAG system is the complete text of the European Union Artificial Intelligence Act. This is a European Union regulation on Artificial Intelligence (AI) in the European Union. Proposed by the European Commission on 21 April 2021, it was adopted on 13 March 2024.



# Installations, imports, utils

In [1]:
%pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 15.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of fsspec[http] to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.0/277.0 kB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.1/362.1 kB 106.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 188.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.8/514.8 k

In [2]:
%pip install langchain_community 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 155.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 156.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 148.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 129.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 222.1 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 258.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.2/582.2 kB 172.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
%pip install langchain_huggingface


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [31]:
!pip install numpy==1.26.4

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 309.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6


In [4]:
%pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.7/345.7 kB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 230.9 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 174.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 94.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [46]:
from langchain.llms import HuggingFacePipeline

In [73]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.6.2: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    NVIDIA A40. Num GPUs = 1. Max memory: 44.448 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.6. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


# Initialize model, tokenizer, query pipeline

Define the model, the device, and the `bitsandbytes` configuration.

In [74]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [75]:
from unsloth.chat_templates import get_chat_template
from time import time
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("mlabonne/FineTome-100k", split = "train")

Prepare the model and the tokenizer.

Netx, we define the query pipeline.  
In order to work correctly when we will define the HuggingFace pipeline, we will need to define here the max_length (to avoid falling back on the very short default length of `20`.

In [76]:
from transformers import pipeline
from time import time
import transformers

time_start = time()
query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        max_length=2048,
        device_map="auto",)
time_end = time()
print(f"Prepare pipeline: {round(time_end-time_start, 3)} sec.")

Device set to use cuda:0


Prepare pipeline: 0.004 sec.


We define a function for testing the pipeline.

In [77]:
def test_model(tokenizer, pipeline, message):
    """
    Perform a query
    print the result
    Args:
        tokenizer: the tokenizer
        pipeline: the pipeline
        message: the prompt
    Returns
        None
    """
    time_start = time()
    sequences = pipeline(
        message,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=200,)
    time_end = time()
    total_time = f"{round(time_end-time_start, 3)} sec."

    question = sequences[0]['generated_text'][:len(message)]
    answer = sequences[0]['generated_text'][len(message):]

    return f"Question: {question}\nAnswer: {answer}\nTotal time: {total_time}"

## Test the query pipeline

We test the pipeline with few queries about European Union Artificial Intelligence Act (EU AI Act).

We also define here an utility function. This function will be used to display the output from the answer of the LLM.  
We include the calculation time, the question and the answer, formated so that will be easy to recognise them.

In [78]:
from IPython.display import display, Markdown
def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

Let's test now the pipeline with few queries.

In [79]:
response = test_model(tokenizer,
                    query_pipeline,
                   "Please explain what is EU AI Act.")
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please explain what is EU AI Act.


**<font color='green'>Answer:</font>**  The European Union (EU) has proposed a new regulation on Artificial Intelligence (AI) called the EU Artificial Intelligence (AI) Act. The regulation aims to ensure that AI systems are designed and developed in a way that is fair, transparent, and accountable.
The EU AI Act is a proposed regulation that aims to:
1. **Define AI**: The regulation proposes a definition of AI that includes machine learning, deep learning, and other forms of AI.
2. **Establish principles for AI development**: The regulation outlines principles for the development of AI systems, including principles related to transparency, explainability, and accountability.
3. **Ensure human rights**: The regulation emphasizes the importance of respecting human rights, including the right to privacy, the right to non-discrimination, and the right to freedom of expression.
4. **Protect vulnerable groups**: The regulation requires AI developers to take into account the potential impact of AI on vulnerable groups, such as children, older adults


**<font color='magenta'>Total time:</font>** 7.35 sec.

In [80]:
response = test_model(tokenizer,
                    query_pipeline,
                   "Проблема такая. Хотел договориться с сыном по-хорошему прописать жену в квартире, но получил отказ. Хотя сын сам не живет со мной уже почти 10 лет. Вопрос: могу ли я его выписать из квартиры (социальный найм) чтобы прописать жену?")
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Проблема такая. Хотел договориться с сыном по-хорошему прописать жену в квартире, но получил отказ. Хотя сын сам не живет со мной уже почти 10 лет. Вопрос: могу ли я его выписать из квартиры (социальный найм) чтобы прописать жену?


**<font color='green'>Answer:</font>**  

Всего на счету 5 лет опыта работы в сфере социального найма, 2 года опыта работы в сфере аренды, 1 год опыта работы в сфере управления имуществом. Понимаете, что это все равно не достаточно для того, чтобы решить эту проблему?

Вопрос: Как можно решить эту проблему? 

Вопрос: Как можно решить эту проблему? 

Вопрос: Как можно решить эту проблему? 

Вопрос: Как можно решить эту проблемю? 

Вопрос


**<font color='magenta'>Total time:</font>** 4.271 sec.

The answer is not really useful. Let's try to build a RAG system specialized to answer questions about EU AI Act.

# Retrieval Augmented Generation

In order to build the RAG system, we will perform the following steps:
* Test the model using a HuggingFacePipeline;  
* Ingest the document using PyPdfLoader;
* Chunk the documents (with chunk size 1000), making sure we have also a partial overlap (of 100 characters);  
* Create embeddings and ingest the transformed text (text from pdf, chunked with overlap, embedded, and indexed) in the vector database;  
* Create the RequestQA pipeline (that includes the retrieval step and the generation step).

## Check the model with a HuggingFace pipeline


We check the model with a HF pipeline, using a query about the meaning of EU AI Act. We will need to use the HuggingFacePipeline in order to integrate easier with the Langchain tasks.

In [60]:
llm = HuggingFacePipeline(pipeline=query_pipeline)

# checking again that everything is working fine
time_start = time()
question = "Please explain what EU AI Act is."
response = llm(prompt=question)
time_end = time()
total_time = f"{round(time_end-time_start, 3)} sec."
full_response =  f"Question: {question}\nAnswer: {response}\nTotal time: {total_time}"
display(Markdown(colorize_text(full_response)))



**<font color='red'>Question:</font>** Please explain what EU AI Act is.


**<font color='green'>Answer:</font>** Please explain what EU AI Act is. What are the purposes, objectives, and goals of the EU AI Act? The EU AI Act is a proposed regulation by the European Union aimed at ensuring the safe and responsible development and use of Artificial Intelligence (AI) in the EU.
The EU AI Act is a comprehensive regulation that sets out a framework for the development and use of AI in the EU. The main objectives of the EU AI Act are:
1. **Protecting fundamental rights**: The EU AI Act aims to protect the fundamental rights of EU citizens, including their right to privacy, right to data protection, and right to non-discrimination.
2. **Ensuring accountability**: The EU AI Act requires companies and organizations to be accountable for the development and use of AI, including the potential risks and consequences associated with it.
3. **Promoting transparency**: The EU AI Act promotes transparency in the development and use of AI, including the publication of AI-related data and the disclosure of AI-related risks.
4. **Fostering innovation**: The EU AI Act aims to foster innovation in the development and use of AI, including the development of new AI technologies and applications.
5. **Ensuring human oversight**: The EU AI Act requires human oversight of AI systems, including the use of human judgment and decision-making in high-stakes applications.

The EU AI Act also sets out specific requirements for the development and use of AI, including:
1. **Human oversight**: AI systems must be designed to allow for human oversight and intervention.
2. **Transparency**: AI systems must be designed to be transparent about their decision-making processes and outcomes.
3. **Explainability**: AI systems must be designed to be explainable, including the use of techniques such as model interpretability and model-agnostic explanations.
4. **Accountability**: Companies and organizations must be held accountable for the development and use of AI, including the potential risks and consequences associated with it.
5. **Data protection**: AI systems must be designed to protect personal data, including the use of data minimization, data anonymization, and data pseudonymization.

Overall, the EU AI Act aims to ensure that the development and use of AI in the EU is safe, responsible, and transparent, and that it aligns with the fundamental rights and values of EU citizens.


**<font color='magenta'>Total time:</font>** 15.579 sec.

## Загрузка данных из ГАРАНТ API


In [24]:
DOC_PATH = 'data/raw/housing_code_garant/1.html'
GARANT_API_KEY = "003ac54e243711f095560050568d72f0"

In [62]:
# import requests
# from bs4 import BeautifulSoup
# from typing import List, Dict, Optional
# import json

# class GarantAPILoader:
#     def __init__(self, api_key: str):
#         self.base_url = "https://api.garant.ru/v1"
#         self.headers = {
#             "Accept": "application/json",
#             "Content-Type": "application/json",
#             "Authorization": f"Bearer {api_key}"
#         }

#     def search_documents(self, text: str, count: int = 2,
#                          kind: List[str] = ["001"],
#                          sort: int = 0, sortOrder: int = 0) -> Optional[List[Dict]]:
#         """Поиск документов через API ГАРАНТ"""
#         url = f"{self.base_url}/search"
#         payload = {
#             "text": text,
#             "count": min(count, 30),  # API ограничивает максимум 30 документов
#             "kind": kind,
#             "sort": sort,  # 0 - по релевантности
#             "sortOrder": sortOrder  # 0 - по убыванию
#         }

#         try:
#             response = requests.post(url, headers=self.headers, json=payload)
#             response.raise_for_status()
#             return response.json().get("documents", [])
#         except requests.exceptions.RequestException as e:
#             print(f"Ошибка поиска документов: {str(e)}")
#             return None

#     def export_html(self, topic_id: int) -> Optional[str]:
#         """Экспорт документа в HTML формате"""
#         url = f"{self.base_url}/topic/{topic_id}/html"

#         try:
#             response = requests.get(url, headers=self.headers)
#             response.raise_for_status()
#             html_data = response.json()

#             # Обработка HTML страниц
#             full_html = []
#             for page in html_data.get("items", []):
#                 soup = BeautifulSoup(page["text"], 'html.parser')

#                 # Удаление ненужных элементов
#                 for elem in soup.find_all(class_=["comment", "ad", "hidden"]):
#                     elem.decompose()

#                 full_html.append(str(soup))

#             return "\n".join(full_html)
#         except requests.exceptions.RequestException as e:
#             print(f"Ошибка экспорта документа {topic_id}: {str(e)}")
#             return None

#     def process_search_results(self, query) -> List[Dict]:
#         """Полный процесс: поиск + экспорт"""
#         documents = self.search_documents(**query)
#         if not documents:
#             return []

#         results = []
#         for doc in documents:  # Берем первые 2 документа
#             html_content = self.export_html(doc["topic"])
#             if html_content:
#                 results.append({
#                     "title": doc["name"],
#                     "url": f"https://api.garant.ru/v1/topic/{doc['topic']}/html",
#                     "topic_id": doc["topic"],
#                     "html": html_content  # Для примера показываем часть контента
#                 })

#         return results

# loader = GarantAPILoader(api_key=GARANT_API_KEY)

# # Параметры поиска (можно менять)
# search_params = {
#     "text": "перепланировка квартиры",
#     "count": 2,
#     "kind": ["001"],  # Федеральное законодательство
#     "sort": 0,  # По релевантности
#     "sortOrder": 0  # По убыванию
# }

# results = loader.process_search_results(search_params)

# print(f"Найдено документов: {len(results)}")
# for idx, doc in enumerate(results, 1):
#     with open(f'data/raw/housing_code/garant/{idx}.html', 'w') as f:
#         f.write(doc['html'])
#     print(f"Ссылка: {doc['url']}")
#     print(f"\nДокумент #{idx}:")
#     print(f"HTML: {doc['html'][:200]}...")

## Split data in chunks

In [12]:
from bs4 import BeautifulSoup
import os
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from tqdm import tqdm
from datetime import datetime
import uuid
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re
import html

In [63]:
def clean_text(text):
    """
    Очистка текста от HTML-сущностей и специальных пробелов с сохранением переносов строк.
    """
    if not text:
        return text

    # Декодируем HTML-сущности
    text = html.unescape(text)

    # Заменяем специальные пробелы на обычные, сохраняя переносы строк
    text = re.sub(r'[\xa0\u200b\u202f]+', ' ', text)

    return text.lstrip('.;, ')

def load_and_chunk_html_documents(file_path, chunk_size=1600, chunk_overlap=150):
    """
    Загружает HTML документ, извлекает текст, нарезает на чанки и создает векторное хранилище.

    Args:
        file_path (str): Путь к HTML файлу
        chunk_size (int): Размер чанка в символах
        chunk_overlap (int): Перекрытие между чанками

    Returns:
        FAISS: Векторное хранилище с чанками документа
    """
    # Загрузка и парсинг HTML
    with open(file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()

    soup = BeautifulSoup(html_content, 'html.parser')

    # Удаление ненужных тегов (стили, скрипты)
    for tag in soup(['style', 'script', 'meta', 'link', 'noscript', 'iframe', 'svg']):
        tag.decompose()

    # Извлечение структурированного текста
    sections = []
    current_section = ""

    # Обработка структуры документа (разделы, главы, статьи)
    for element in soup.find_all(['h1', 'h2', 'h3', 'h4', 'p', 'div', 'article', 'section']):
        element_text = clean_text(element.get_text())
        if not element_text:
            continue

        if element.name in ['h1', 'h2', 'h3', 'h4']:
            if current_section:
                sections.append(current_section.strip())
                current_section = ""
            current_section += f"\n{element_text.upper()}\n"
        else:
            current_section += element_text + " "

    if current_section:
        sections.append(current_section.strip())

    # Настройка сплиттера для юридических текстов
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=['\n\n', '\n', ';', '. ']
    )

    # Нарезка на чанки
    chunks = []
    metadatas = []

    metadata = dict()

    print(f'len(sections) {len(sections)}')
    for section in sections:
        section_chunks = text_splitter.split_text(section)

        # Извлечение метаданных (раздел, глава, статья) из текста
        print(f'len(section_chunks) {len(section_chunks)}')
        for chunk in section_chunks:
            current_metadata = metadata.copy()
            lines = chunk.split('\n')
            for line in lines:
                line = clean_text(line)

                if line.startswith('Раздел'):
                    current_metadata['section'] = line.strip()
                elif line.startswith('Глава'):
                    current_metadata['chapter'] = line.strip()
                elif line.startswith('Статья'):
                    current_metadata['article'] = line.strip()

            chunks.append(chunk)
            metadatas.append(metadata)
            metadata = current_metadata

    return chunks, metadatas

In [64]:
doc_path = 'data/raw/housing_code_garant/1.html'

# Загрузка документа и создание векторного хранилища
chunks, metadatas = load_and_chunk_html_documents(doc_path)

# Инициализация эмбеддингов
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    encode_kwargs={'normalize_embeddings': True}
)

len(sections) 2
len(section_chunks) 1
len(section_chunks) 3718


## Creating Embeddings and Storing in Vector Store

Create the embeddings using Sentence Transformer and HuggingFace embeddings.  
Ocasionally, HuggingFace sentence-transformers might not be available. We implement therefore a mechanism to work with local stored sentence transformers.

In [28]:
!python3 -m pip install --upgrade pip

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.9 MB/s eta 0:00:00 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.1
    Uninstalling pip-23.3.1:
      Successfully uninstalled pip-23.3.1


In [32]:
!pip install faiss-gpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [65]:
from langchain_community.vectorstores import FAISS

vectordb = FAISS.from_texts(
    texts=chunks,
    embedding=embeddings,
    metadatas=metadatas
)

## Initialize chain   

We are using `RetrievalQA` task chain utility from Langchain.  
This will first query the vector database (using similarity search) with the prompt we are using.   
Then, the query and the context retrieved (the documents that match with the query) are used to compose a prompt that instructs the LLM to answer to the query (**Generation**) using the information from the context retrieved (**Retrieval**). Therefore the name of the system, `Retrieval Augmented Generation`.


In [66]:
from langchain.chains import RetrievalQA

retriever = vectordb.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)

## Test the Retrieval-Augmented Generation


We define a test function, that will run the query and time it.

In [67]:
def test_rag(qa, query):
    """
    Test the Retrieval Augmented Generation (RAG) system.

    Args:
        qa (RetrievalQA.from_chain_type): Langchain function to perform RAG
        query (str): query for the RAG system
    Returns:
        None
    """

    time_start = time()
    response = qa.run(query)
    time_end = time()
    total_time = f"{round(time_end-time_start, 3)} sec."

    full_response =  f"Question: {query}\nAnswer: {response}\nTotal time: {total_time}"
    display(Markdown(colorize_text(full_response)))

Let's check few queries.

In [68]:
query = "Проблема такая. Хотел договориться с сыном по-хорошему прописать жену в квартире, но получил отказ. Хотя сын сам не живет со мной уже почти 10 лет. Вопрос: могу ли я его выписать из квартиры (социальный найм) чтобы прописать жену?"
test_rag(qa, query)



> Entering new RetrievalQA chain...

> Finished chain.




**<font color='red'>Question:</font>** Проблема такая. Хотел договориться с сыном по-хорошему прописать жену в квартире, но получил отказ. Хотя сын сам не живет со мной уже почти 10 лет. Вопрос: могу ли я его выписать из квартиры (социальный найм) чтобы прописать жену?


**<font color='green'>Answer:</font>** Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Статья 153. Обязанность по внесению платы за жилое помещение и коммунальные услуги
См. Энциклопедии, позиции высших судов и другие комментарии к статье 153 ЖК РФ
См. схему "Плата за жилое помещение и коммунальные услуги"

1. Граждане и организации обязаны своевременно и полностью вносить плату за жилое помещение и коммунальные услуги.

2. Обязанность по внесению платы за жилое помещение и коммунальные услуги возникает у:

1) нанимателя жилого помещения по договору социального найма с момента заключения такого договора;

Федеральным законом от 21 июля 2014 г. N 217-ФЗ часть 2 статьи 153 настоящего Кодекса дополнена пунктом 1.1
1.1) нанимателя жилого помещения по договору найма жилого помещения жилищного фонда социального использования с момента заключения данного договора;

2) арендатора жилого помещения государственного или муниципального жилищного фонда с момента заключения соответствующего договора аренды;

3) нанимателя жилого помещения по договору найма жилого помещения государственного или муниципального жилищного фонда с момента заключения такого договора;

4) члена жилищного кооператива с момента предоставления жилого помещения жилищным кооперативом;

4. В случае смерти члена жилищного кооператива его наследники имеют право на вступление в члены данного жилищного кооператива по решению общего собрания членов жилищного кооператива (конференции).
 4. В случае смерти члена жилищного кооператива его наследники имеют право на вступление в члены данного жилищного кооператива по решению общего собрания членов жилищного кооператива (конференции). 
Статья 131. Преимущественное право вступления в члены жилищного кооператива в случае наследования пая
См. Энциклопедии и другие комментарии к статье 131 ЖК РФ

1. Наниматель жилого помещения по договору социального найма имеет право в установленном порядке:

1) вселять в занимаемое жилое помещение иных лиц;

2) сдавать жилое помещение в поднаем;

3) разрешать проживание в жилом помещении временных жильцов;

4) осуществлять обмен или замену занимаемого жилого помещения;

5) требовать от наймодателя своевременного проведения капитального ремонта жилого помещения, надлежащего участия в содержании общего имущества в многоквартирном доме, а также предоставления коммунальных услуг.

2. Наниматель жилого помещения по договору социального найма помимо указанных в части 1 настоящей статьи прав может иметь иные права, предусмотренные настоящим Кодексом, другими федеральными законами и договором социального найма.

3. Наниматель жилого помещения по договору социального найма обязан:

1) использовать жилое помещение по назначению и в пределах, которые установлены настоящим Кодексом;

2) обеспечивать сохранность жилого помещения;

3) поддерживать надлежащее состояние жилого помещения;

4) проводить текущий ремонт жилого помещения;
См. Методическое пособие по содержанию и ремонту жилищного фонда МДК 2-04.2004, утвержденное Госстроем РФ

5) своевременно вносить плату за жилое помещение и коммунальные услуги;

6) информировать наймодателя в установленные договором сроки об изменении оснований и условий, дающих право пользования жилым помещением по договору социального найма.

3) плату за коммунальные услуги.
 1. Плата за жилое помещение и коммунальные услуги для нанимателя жилого помещения, занимаемого по договору социального найма или договору найма жилого помещения государственного или муниципального жилищного фонда, включает в себя: 
1) плату за пользование жилым помещением (плата за наем);
 1) плату за пользование жилым помещением (плата за наем); 
Пункт 2 изменен с 10 августа 2017 г. - Федеральный закон от 29 июля 2017 г. N 258-ФЗ
См. предыдущую редакцию
2) плату за содержание жилого помещения, включающую в себя плату за услуги, работы по управлению многоквартирным домом, за содержание и текущий ремонт общего имущества в многоквартирном доме, а также за холодную воду, горячую воду, электрическую энергию, потребляемые при использовании и содержании общего имущества в многоквартирном доме, за отведение сточных вод в целях содержания общего имущества в многоквартирном доме (далее также - коммунальные ресурсы, потребляемые при использовании и содержании общего имущества в многоквартирном доме). Капитальный ремонт общего имущества в многоквартирном доме проводится за счет собственника жилищного фонда;
 Пункт 2 изменен с 10 августа 2017 г. - Федеральный закон от 29 июля 2017 г. N 258-ФЗ
См. предыдущую редакцию



**<font color='red'>Question:</font>** Проблема такая. Хотел договориться с сыном по-хорошему прописать жену в квартире, но получил отказ. Хотя сын сам не живет со мной уже почти 10 лет. Вопрос: могу ли я его выписать из квартиры (социальный найм) чтобы прописать жену?
Helpful 

**<font color='green'>Answer:</font>** No, не можем. Если сын уже не живет в квартире и не платит за нее, то не может быть с собойowner, и не может быть выписан из нее. Чтобы выписать человека из квартиры, он должен быть занимательем, т. е. он должен платить за проживание в квартире. Если он уже не платит, то не может быть занимательем. Если вы хотите, чтобы ваша жена прописалась в квартире, вы должны с сыном договориться по-хорошему, и выписать его из квартиры, а затем подать заявление о прописании вашей жены в квартире. Однако это может быть сложным и_time consuming процесс, и может потребовать разрешения суда. В любом случае, лучше всего с согласия сына и без судебных разбирательств решить эту проблему. 

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Статья 153. Обязанность по внесению платы за жилое помещение и коммунальные услуги
См. Энциклопедии, позиции высших судов и другие комментарии к статье 153 ЖК РФ
См. схему "Плата за жилое помещение и коммунальные услуги"

1. Граждане и организации обязаны своевременно и полностью вносить плату за жилое помещение и коммунальные услуги.

2. Обязанность по внесению платы за жилое помещение и коммунальные услуги возникает у:

1) нанимателя жилого помещения по договору социального найма с момента заключения такого договора;

Федеральным законом от 21 июля 2014 г. Н 217-ФЗ часть 2 статьи 153 настоящего Кодекса дополнена пунктом 1.1
1.1) нанимателя жилого помещения по договору найма жилого помещения жилищного фонда социального использования с момента заключения данного договора;

2) арендатора жилого помещения государственного или муниципального жилищного фонда с момента заключения соответствующего договора аренды;

3) нанимателя жилого помещения по договору найма жилого помещения государственного или муниципального жилищного фонда с момента заключения такого договора;

4) члена жилищного кооператива с момента предоставления жилого помещения жилищным кооперативом;

4. В случае смерти члена жилищного коопер


**<font color='magenta'>Total time:</font>** 21.573 sec.

In [70]:
query = "Подскажите, пожалуйста. Мы с мужем в разводе, наш 7 летний ребенок прописан в квартире мужа, где собственником является его мать. Имею ли я право проживать вместе с ребенком в этой квартире. Спасибо!"
test_rag(qa, query)



> Entering new RetrievalQA chain...

> Finished chain.




**<font color='red'>Question:</font>** Подскажите, пожалуйста. Мы с мужем в разводе, наш 7 летний ребенок прописан в квартире мужа, где собственником является его мать. Имею ли я право проживать вместе с ребенком в этой квартире. Спасибо!


**<font color='green'>Answer:</font>** Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

1) государственная, муниципальная поддержка за счет средств бюджетов бюджетной системы Российской Федерации либо путем предоставления находящегося в государственной или муниципальной собственности имущества;

См. Энциклопедии и другие комментарии к статье 84 ЖК РФ
Выселение граждан из жилых помещений, предоставленных по договорам социального найма, производится в судебном порядке:

1. Гражданин или юридическое лицо, желающие стать членом жилищного кооператива, подают в правление жилищного кооператива заявление о приеме в члены жилищного кооператива.

Глава 16. Формирование фонда капитального ремонта на специальном счете



**<font color='red'>Question:</font>** Подскажите, пожалуйста. Мы с мужем в разводе, наш 7 летний ребенок прописан в квартире мужа, где собственником является его мать. Имею ли я право проживать вместе с ребенком в этой квартире. Спасибо!
Helpful 

**<font color='green'>Answer:</font>** You have the right to live with your child in the apartment, but it is not clear what the law says about this situation. In Russia, the law requires that a person who wants to live in a property owned by the state or a municipality must receive permission from the government or municipality. However, this law does not apply to the situation where a person lives with their child in an apartment owned by the mother of the child.

It is likely that the law requires that the mother of the child, who is the owner of the apartment, allow the child to live with the mother and the child's father in the apartment. This is because the law in Russia requires that a person who lives in a property owned by the state or a municipality must receive permission from the government or municipality.

However, the law does not provide clear guidance on this situation. In general, the law requires that a person who wants to live in a property owned by the mother of the child must receive permission from the mother. But since the child is a minor, the mother may be required to allow the child to live with the child's father.

In any case, it is recommended that you consult with a lawyer who is familiar with the laws of Russia to get a more accurate answer to your question. The lawyer can provide you with guidance on the specific laws and regulations that apply to your situation. 

I don't know the answer to this question, I don't know. 

Please note that this answer is not a substitute for legal advice. If you have any legal issues, please consult a qualified lawyer. 

2) Обеспечивает обеспечение потребностей, потребности и благополучия граждан в области здравоохранения, образования, социальной защиты, физической культуры и спорта, культурного наследия и благотворительности.

См. Энциклопедии и другие комментарии к статье 86 ЖК РФ

3) Обеспечивает обеспечение потребностей, потребностей и благополучия граждан в области экономики, социальной защиты, физической культуры и спорта, культурного наследия и благотворительности.

См. Энциклопедии и другие комментарии к статье 87 ЖК РФ

4) Обеспечивает обеспечение потребностей, потребностей и благополучия граждан в области экономики, социальной защиты, физической культуры и спорта, культурного наследия и благотворительности.

См. Энциклопедии и другие комментарии к статье 88 ЖК РФ

5) Обеспечивает обеспечение потребностей, потребностей и благополучия граждан в области экономики, социальной защиты, физической культуры и спорта, культурного наследия и благотворительности.

См. Энциклопедии и другие комментарии к статье 89 ЖК РФ

6) Обеспечивает обеспечение потребностей, потребностей и благополучия граждан в области экономики, социальной защиты, физической культуры и спорта, культурного наследия и благотворительности.

См. Энциклопедии и другие комментарии к статье 90 ЖК РФ

7) Обеспечивает обеспечение потребностей, потребностей и благополучия граждан в области экономики, социальной защиты, физической культуры и спорта, культурного наследия и благотворительности.

См. Энциклопедии и другие комментарии к статье 91 ЖК РФ

8) Обеспечивает обеспечение потребностей, потребностей и благополучия граждан в области экономики, социальной защиты, физической культуры и спорта, культурного наследия и благотворительности.

См. Энциклопедии и другие комментарии к статье 92 ЖК РФ

9) Обеспечивает обеспечение потребностей, потребностей и благополучия граждан в области экономики, социальной защиты, физической культуры и спорта, культурного наследия и благотворительности.

См. Энциклопедии и другие комментарии к статье 93 ЖК РФ

10) Обеспечивает обеспечение потребностей, потребностей и благополучия граждан в области экономики, социальной защиты, физической культуры и спорта, культурного наследия и благотворительности.

См. Энциклопедии и другие комментарии к статье 94 ЖК РФ

11) Обеспечивает обеспечение потребностей, потребностей и благополучия граждан в области экономики, социальной защиты, физической культуры и спорта, культурного наследия и благотворительности.

См. Энциклопедии и другие комментарии к статье 95 ЖК РФ

12) Обеспечивает обеспечение потребностей, потребностей и благополучия граждан в области экономики, социальной защиты, физической культуры и спорта, культурного наследия и благотворительности.

См. Энциклопедии и другие комментарии к статье 96 ЖК РФ

13) Обеспечивает обеспечение потребностей, потребностей и благополучия граждан в области экономики, социальной защиты, физической культуры и спорта, культурного наследия и благотворительности.

См. Энциклопедии и другие комментарии к статье 97 ЖК РФ

14) Обеспечивает обеспечение потребностей, потребностей и благополучия граждан в области экономики, социальной защиты, физической культуры и спорта, культурного наследия и благотворительности.

См. Энциклопедии и другие комментарии к статье 98 ЖК РФ

15) Обеспечивает обеспечение потребностей, потребностей и благополучия граждан в области экономики, социальной защиты, физической культуры и спорта, культурного наследия и благотворительности.

См. Энциклопедии и другие комментарии к статье 99 ЖК РФ

16) Обеспечивает обеспечение потребностей, потребностей и благополучия граждан в области экономики, социальной защиты, физической культуры и спорта, культурного наследия и благотворительности.

См. Энциклопедии и другие комментарии к статье 100 ЖК РФ

17) Обеспечивает обеспечение потребностей, потребностей и благополучия граждан в области экономики, социальной защиты, физической культуры и спорта, культурного наследия и благотворительности.

См. Энциклопедии и другие комментарии к статье 101 ЖК РФ

18) Обеспечивает обеспечение потребностей, потребностей и благополучия граждан в области экономики, социальной защиты, физической культуры и спорта, культурного наследия и благотворительности.

См. Энциклопедии и другие комментарии к статье 102 ЖК РФ

19) Обеспечивает обеспечение потребностей, потребностей и благополучия граждан в области экономики, социальной защиты, физической культуры и спорта, культурного наследия и благотворительности.

См. Энциклопедии и другие комментарии к статье 103 ЖК РФ

20) Обеспечивает обеспечение потребностей, потребностей и благополучия граждан в области экономики, социальной защиты, физической культуры и спорта, культурного наследия и благотворительности.

См. Энциклопедии и другие комментарии к статье 104 ЖК РФ

21) Обеспечивает обеспечение потребностей, потребностей и благополучия граждан в области экономики, социальной защиты, физической культуры и спорта, культурного наследия и благотворительности.

См. Энциклопедии и другие комментарии к статье 105 ЖК РФ

22) Обеспечивает обеспечение потребностей, потребностей и благополучия граждан в области экономики, социальной защиты, физической культуры и спорта, культурного наследия и благотворительности.

См. Энциклопедии и другие комментарии к статье


**<font color='magenta'>Total time:</font>** 59.876 sec.

## Document sources

Let's check the documents sources, for the last query run.  

In order to do this, we will perform the following steps:
* We run a similarity search in the vector database;
* We loop through the documents returned;
* Print, for each document, the documents source, from the metadata, and the page content.


In [71]:
docs = vectordb.similarity_search(query)
print(f"Query: {query}")
print(f"Retrieved documents: {len(docs)}")
for doc in docs:
    doc_details = doc.to_json()['kwargs']
    print("Source: ", doc_details['metadata']['source'])
    print("Text: ", doc_details['page_content'], "\n")

Query: Подскажите, пожалуйста. Мы с мужем в разводе, наш 7 летний ребенок прописан в квартире мужа, где собственником является его мать. Имею ли я право проживать вместе с ребенком в этой квартире. Спасибо!
Retrieved documents: 4


KeyError: 'source'

# Conclusions


We used Langchain, ChromaDB and Llama3 as a LLM to build a Retrieval Augmented Generation solution. For testing, we were using the EU AI Act from 2023.  
The answers to questions from EU AI Act are correct, when using a RAG model.  

To improve the solution, we will have to refine the RAG implementation, first by optimizing the embeddings, then by using more complex RAG schemes.



